In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/undersampledrenn/Repeated_Edited_Nearest_Neighbors_Under-sampled_Dataset.csv


In [5]:
import pandas as pd
import numpy as np
df = pd.read_csv('/kaggle/input/undersampledrenn/Repeated_Edited_Nearest_Neighbors_Under-sampled_Dataset.csv')
df = df[df['Label'] != 'BENIGN']

# Startified K-fold

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Dropout
from keras.utils import np_utils

# Extract the features and label
features = df.drop("Label", axis=1).values
labels = df["Label"].values

# Encode the labels to numeric values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = np_utils.to_categorical(labels)

# Define the number of folds for stratified K-fold cross-validation
k_folds = 5

# Define the RNN model
model = Sequential()
model.add(SimpleRNN(100, input_shape=(features.shape[1], 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(100))
model.add(Dropout(0.2))
model.add(Dense(labels.shape[1], activation="softmax"))

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

train_acc_list = []
train_loss_list = []
test_acc_list = []
test_loss_list = []
# Perform stratified K-fold cross-validation
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)
for i, (train_index, test_index) in enumerate(skf.split(features, labels.argmax(1))):
    print("Fold", i+1)

    # Split the data into training and testing sets
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    # Reshape the data for RNN
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # Train the model
    model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

    # Evaluate the model
    train_loss, train_accuracy = model.evaluate(X_train, y_train)

    # Evaluate the model on the test set
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    
    train_acc_list.append(train_accuracy)
    train_loss_list.append(train_loss)
    test_acc_list.append(test_accuracy)
    test_loss_list.append(test_loss)

print("Average Training Accuracy: {}".format(np.mean(train_acc_list)))
print("Average Training Loss: {}".format(np.mean(train_loss_list)))
print("Average Testing Accuracy: {}".format(np.mean(test_acc_list)))
print("Average Testing Loss: {}".format(np.mean(test_loss_list)))

Fold 1
Epoch 1/5
5000/5000 [==============================] - 337s 67ms/step - loss: 0.5459 - accuracy: 0.7979 - val_loss: 0.2899 - val_accuracy: 0.8920
Epoch 2/5
5000/5000 [==============================] - 334s 67ms/step - loss: 0.3103 - accuracy: 0.8870 - val_loss: 0.2671 - val_accuracy: 0.8970
Epoch 3/5
5000/5000 [==============================] - 335s 67ms/step - loss: 0.3044 - accuracy: 0.8886 - val_loss: 0.3386 - val_accuracy: 0.8834
Epoch 4/5
5000/5000 [==============================] - 334s 67ms/step - loss: 0.2911 - accuracy: 0.8923 - val_loss: 0.3068 - val_accuracy: 0.8787
Epoch 5/5
1250/1250 [==============================] - 10s 8ms/step - loss: 0.2609 - accuracy: 0.9014
Fold 2
Epoch 1/5
5000/5000 [==============================] - 333s 67ms/step - loss: 0.3100 - accuracy: 0.8882 - val_loss: 0.2736 - val_accuracy: 0.8981
Epoch 2/5
5000/5000 [==============================] - 334s 67ms/step - loss: 0.3019 - accuracy: 0.8867 - val_loss: 0.2692 - val_accuracy: 0.8917
Epoch 3/

# Just the simple model

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Dropout
from keras.utils import np_utils

# Extract the features and label
features = df.drop("Label", axis=1).values
labels = df["Label"].values

# Encode the labels to numeric values
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)
labels = np_utils.to_categorical(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Reshape the data for RNN
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Define the RNN model
model = Sequential()
model.add(SimpleRNN(100, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(100))
model.add(Dropout(0.2))
model.add(Dense(labels.shape[1], activation="softmax"))

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


Epoch 1/5
2775/2775 [==============================] - 127s 45ms/step - loss: 0.5717 - accuracy: 0.7953 - val_loss: 0.3021 - val_accuracy: 0.8789
Epoch 2/5
2775/2775 [==============================] - 125s 45ms/step - loss: 0.3463 - accuracy: 0.8653 - val_loss: 0.3123 - val_accuracy: 0.8737
Epoch 3/5
2775/2775 [==============================] - 125s 45ms/step - loss: 0.3752 - accuracy: 0.8568 - val_loss: 0.3205 - val_accuracy: 0.8721
Epoch 4/5
2775/2775 [==============================] - 125s 45ms/step - loss: 0.3330 - accuracy: 0.8697 - val_loss: 0.3637 - val_accuracy: 0.8620
Epoch 5/5
2775/2775 [==============================] - 124s 45ms/step - loss: 0.3665 - accuracy: 0.8611 - val_loss: 0.3181 - val_accuracy: 0.8746
Test loss: 0.3181447386741638
Test accuracy: 0.8746029734611511


Use optuna